In [1]:
import sys
import os
curPath = os.path.abspath(os.path.dirname("sequential_model.py"))
print(curPath)
rootPath = os.path.split(curPath)[0]
print(rootPath)
sys.path.append(rootPath)
sys.path.append(curPath)
import torch
from utils import datasets as ds
from utils.transforms import SpatialTransformer, CameraTransform
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import cv2
import sequential_model
import warnings
import numpy as np
import random
warnings.filterwarnings("ignore")

/homes/ql1220/PoseEstimate/infant-3d-pose-estimation/core
/homes/ql1220/PoseEstimate/infant-3d-pose-estimation


/vol/bitbucket/ql1220/envs/pose_env/lib/python3.8/site-packages/spacepy/time.py:2294: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'
/vol/bitbucket/ql1220/envs/pose_env/lib/python3.8/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(
/vol/bitbucket/ql1220/envs/pose_env/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:569: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.object, string),
/vol/bitbucket/ql1220/envs/pose_env/lib/python3.8/site-packages/tensor

In [2]:
activities = ['directions']
actors=['S%d' % i for i in [1]]
trans=transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor()
    ])
# root='/vol/biodata/data/h36m/training'
root='/vol/biodata/data/h36m/training'
pose_root='/vol/biodata/data/h36m/h36m_poses/d2'
h=ds.Human36M(root,pose_root,activities=activities, actors=actors,sequence_len=20,random_seed=2,transforms=trans, is_train=True, subsampled_size=None)

100%|██████████| 1/1 [00:02<00:00,  2.64s/it]


In [3]:
pose3d_root='/vol/biodata/data/h36m/h36m_poses/d3'
pose3d=ds.Human36MPose(pose3d_root,h.get_sequences())
a,b,c=h.get_single(h.sequences[0])
print(a.size(),b.size(),c.size())
p1=pose3d.get_single(h.sequences[0])
print(p1.size())
poseloader = DataLoader(pose3d,batch_size=10,shuffle=True)

torch.Size([3, 20, 256, 256]) torch.Size([3, 256, 256]) torch.Size([16, 2, 20])
torch.Size([16, 3, 20])


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.device_count(),torch.cuda.current_device(),torch.cuda.get_device_capability(device),  torch.cuda.get_device_name(device))

cuda
1 0 (6, 1) GeForce GTX 1080


In [4]:
torch.cuda.empty_cache()
model=sequential_model.ForwardKinematics(h,pose3d)
logdir='/vol/bitbucket/ql1220/out/log_test1'
checkdir='/vol/bitbucket/ql1220/out/check_test1'

In [5]:
model.train(1, 5, logdir, checkdir)

epoch: 0
50
100
150
200
250
epoch: 1
300
350
400
450
500
550
epoch: 2
600
650
700
750
800
epoch: 3
850
900
950
1000
1050
1100
epoch: 4
1150
1200
1250
1300
1350


In [8]:
import matplotlib.pyplot as plt

In [5]:
batch_size=1
len_seq=20
dataloader = DataLoader(h, batch_size, shuffle=True, drop_last=True, num_workers=8)
def draw(dataloader,batch_size,model,len_seq):
    with torch.no_grad():
        i=0
        for batch in dataloader:
            frames, frame2, preds2d = batch
            frames, frame2, preds2d = frames.to(model.device), frame2.to(model.device), preds2d.to(model.device)
            preds2d = model.normalize(preds2d).float()
            preds2d1=(preds2d+1)*32
            preds2d1=preds2d1.permute(0, 3, 1, 2).reshape(-1, 16, 2)
            heatmaps = model.transformer(preds2d1.flip(-1))
            heatmaps = model.upsample(heatmaps)

            # convert batch*T, ch, h, w to batch, ch, h, w, T for visualization

            heatmaps1 = heatmaps.view(batch_size, len_seq, 16, 256, 256).permute(0, 2, 3, 4, 1)
            heatmap_grid = model.show_sequence_images(heatmaps1.sum(1, keepdims=True).detach())
            plt.figure(i+1)
            plt.imshow(heatmap_grid.permute(1,2,0).cpu())
            i=i+1
            if i>5:
                break
        plt.show()
        
        


In [4]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

logdir='/vol/bitbucket/ql1220/out/log_test1'
checkdir='/vol/bitbucket/ql1220/out/check_test1'
torch.cuda.empty_cache()
model_test=sequential_model.ForwardKinematics(h,pose3d)
model_test.train(1,1,logdir,checkdir)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [8]:
class Human36MPose(Dataset):
    def __init__(self,root,sequences, sequence_len=20):
        self.root=root
        self.sequences=sequences
        self.seq_len=sequence_len
        
    def get_single(self,sequence):
        pose_folder = os.path.join(self.root, sequence['actor'], sequence['activity_sequence'])
        pose_name='/3d_pose_'
        poses=[]
        for i in sequence['frames']:
            pose=np.load(pose_folder+pose_name+str(i-1)+'.npy')
            poses.append(pose)
        pose_array = np.array(poses)
        pose = pose_array[:,joints,:]#(20, 16, 3)
        pose = self.align(pose)
        first = pose[0]
        pelvis = first[0] + (first[1] - first[0]) / 2
        pose -= pelvis
        pose = self.normalize(pose)
        pose_tensor = torch.FloatTensor(pose)
        pose_tensor = pose_tensor.permute(1,2,0)
        
        return pose_tensor
    
    def align(self, sequence):
        """remove y component of hip line,
        align pelvis-neck line with z axis"""
        points=sequence[0]
        pelvis = points[0] + (points[1] - points[0]) / 2
        neck = points[2] + (points[3] - points[2]) / 2
        pelvis_neck_line = neck - pelvis
        # pelvis neck
        rot_axis1 = np.array([0, -1, 0])
        angle = self.get_angle(pelvis_neck_line, rot_axis1)
        cross_prod = np.cross(pelvis_neck_line, rot_axis1)
        cross_prod /= np.linalg.norm(cross_prod)
        R1 = R.from_rotvec(angle * cross_prod)
        for i in range(len(sequence)):
            sequence[i] = R1.apply(sequence[i])
        # hip
        points=sequence[0]
        hip_line = points[0] - points[1]
        rot_axis2 = np.array([-1, 0])
        indices = np.array([0, 2])
        angle = self.get_angle(hip_line[indices], rot_axis2)
        R2 = R.from_rotvec(np.array([0, angle, 0]))
        for i in range(len(sequence)):
            sequence[i] = R2.apply(sequence[i])

        rot = R.from_rotvec(np.array([0,0, np.pi]))
        for i in range(len(sequence)):
            sequence[i] = rot.apply(sequence[i])

        return sequence
    
    def normalize(self, points):
        first = points[0]
        # normalize to unit cube -1 to 1
        max_ = np.abs(first.max())
        min_ = np.abs(first.min())
        if max_ >= min_:
            points /= max_
        else:
            points /= min_
        return points
    
    def get_angle(self,vec1, vec2):
        inv = vec1 @ vec2 / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        return np.arccos(inv)
        
    def __getitem__(self,index):
        seq=self.sequences[index]
        return  self.get_single(seq)
        
    def __len__(self):
        if self.sequences:
            return len(self.sequences)
        else:
            raise(RuntimeError("Dataset is empty!"))
            return 0